In [1]:
from collections import Counter
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
import fuzzywuzzy
from fuzzywuzzy import fuzz
from  nltk import word_tokenize
import torch.optim as optim
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
import io
import utils

D:\Softwares\Anaconda\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
cuda = torch.device("cuda:0")

In [3]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [4]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [5]:
def get_processed_tokens_for_english_tweets():
    f = "english/agr_en_train.csv"
    # preprocessing english tweets.
    #ingesting english csv file
    df = pd.read_csv(f,names = ['source','comment','annotation'],encoding='UTF-8')
    df['comment'] = df.comment.str.strip()   # removing spaces
    comments = np.asarray(df['comment'])    # dividing the dataframe into comments and tags and converting to array
    tags = np.asarray(df['annotation'])
    print((len(comments)))
    print(len(tags))
    stop_words = set(stopwords.words('english'))  #english stop words list
    processed_tokens = []
    for comment in comments:
    # comment = "Also see ....hw ur RSS activist caught in Burkha .... throwing beef in d holy temples...https://www.google.co.in/amp/www.india.com/news/india/burkha-clad-rss-activist-caught-throwing-beef-at-temple-pictures-go-viral-on-facebook-593154/amp/,NAGfacebook_corpus_msr_403402,On the death of 2 jawans in LOC CROSS FIRING"
        comment = comment.lower()   #lower casing each tweets
        Digit_REMOVAL = re.sub(r'[0-9]+', '',comment) #removal of numbers 
        URL_REMOVAL = re.sub(r"http\S+", "", Digit_REMOVAL) # removal of URLS
        tokenizer = nltk.RegexpTokenizer(r"\w+")   # removal of punctuation and tokenizing
        new_words = tokenizer.tokenize(URL_REMOVAL)
        sentence = []
        for word in new_words:
            if word not in stop_words:           #checking for stop words on each sentence
                sentence.append(word)
        processed_tokens.append(sentence)
    return processed_tokens, tags

In [6]:
#-----------------For hinglish dataset

def get_processed_hindi_tokens():
    Hindi_text  = "hindi/agr_hi_dev.csv"
    df1 = pd.read_csv(Hindi_text,names = ['source','comment','annotation'],encoding='UTF-8')
    df1['comment'] = df1.comment.str.strip()   # removing spaces
    hindi_comments = np.asarray(df1['comment'])    # dividing the dataframe into comments and tags and converting to array
    hindi_tags = np.asarray(df1['annotation'])
    print((hindi_comments[1])) 
    processed_Hindi_tokens = []
    for comment in hindi_comments:
    #   comment = "Also see ....hw ur RSS activist caught in Burkha .... throwing beef in d holy temples...https://www.google.co.in/amp/www.india.com/news/india/burkha-clad-rss-activist-caught-throwing-beef-at-temple-pictures-go-viral-on-facebook-593154/amp/,NAGfacebook_corpus_msr_403402,On the death of 2 jawans in LOC CROSS FIRING"
        comment = comment.lower()   #lower casing each tweets
        Digit_REMOVAL = re.sub(r'[0-9]+', '',comment) #removal of numbers 
        URL_REMOVAL = re.sub(r"http\S+", "", Digit_REMOVAL) # removal of URLS
        Emoji_removal = remove_emoji(URL_REMOVAL)
        if (isEnglish(Emoji_removal) == True):
            Emoji_removal = re.sub(r'[^\w\s]','',Emoji_removal)# removal of punctuation and tokenizing
        processed_Hindi_tokens.append(word_tokenize(Emoji_removal))
    print(processed_Hindi_tokens[0])
    return processed_Hindi_tokens


In [7]:
#-----------Transliteration and translation
def get_transliteration_Hinglish_Hindi_dict():
    transliteration_dict = "transliterations.hi-en.csv"
    t_dict = pd.read_csv(transliteration_dict,names = ['Hinglish','Hindi'],encoding='UTF-8',sep='\t')
    t_dict['Hinglish'] = t_dict['Hinglish'].str.strip()
    t_dict['Hindi'] = t_dict['Hindi'].str.strip()
    print(t_dict)
    t_dict = np.asarray(t_dict)
    print("After NP array")
    print(t_dict)
    return t_dict

#--------------profanity dictionary
def get_profanity_dict():
    profanity_dict = "ProfanityText.txt"
    P_dict = pd.read_csv(profanity_dict,names = ['Hinglish','English'],encoding='UTF-8',sep='\t')
    P_dict['Hinglish'] = P_dict['Hinglish'].str.strip()
    P_dict['English'] = P_dict['English'].str.strip()
    print("Profanity")
    print(P_dict)
    P_dict = np.asarray(P_dict)
    print("After NP array")
    print(P_dict)
    return P_dict

In [8]:
#------------------Translation of hindi text back to english-------
def translate_hindi_to_english():
    Hindi_dict = "Hindi_English_Dict.csv"
    H_dict = pd.read_csv(Hindi_dict,names = ['Hindi','English'],encoding='UTF-8')

    HE_dict_F = "HE_dictionary_functions.csv"
    H_dict_F = pd.read_csv(HE_dict_F,names = ['Hindi','English'],encoding='UTF-8')
    H_dict_F['Hindi'] = H_dict_F['Hindi'].str.strip()
    H_dict_F['English'] = H_dict_F['English'].str.strip()

    H_hindi_F = np.asarray(H_dict_F['Hindi'])
    H_english_F = np.asarray(H_dict_F['English'])

    H_dict['Hindi'] = H_dict['Hindi'].str.strip()
    H_dict['English'] = H_dict['English'].str.strip()

    H_hindi = np.asarray(H_dict['Hindi'])
    H_english = np.asarray(H_dict['English'])
    
    HE_dict = dict(zip(H_hindi,H_english))
    H_dict_F = dict(zip(H_hindi_F,H_english_F))

    EH_dict = {v:k for k, v in HE_dict.items()}
    EH_dict_F = {v:k for k, v in H_dict_F.items()}
    
    return HE_dict, H_dict_F, EH_dict, EH_dict_F

In [9]:
# Convert
def get_translated_hindi_english(HE_dict, H_dict_F):
    for i in range(0,len(processed_Hindi_tokens)):
        #print(i)
        for j in range (0,len(processed_Hindi_tokens[i])):
            Str = processed_Hindi_tokens[i][j]
            if(Str in HE_dict):
                processed_Hindi_tokens[i][j] = HE_dict[Str]
            elif(Str in H_dict_F):
                processed_Hindi_tokens[i][j] = H_dict_F[Str]
    print(processed_Hindi_tokens[0])
    return processed_Hindi_tokens

In [10]:
def get_token_translations(processed_tokens, processed_Hindi_tokens, EH_dict, P_dict, t_dict):
    for i in range(0,len(processed_Hindi_tokens)):
        for j in range (0,len(processed_Hindi_tokens[i])):
            flag = 0
            Str1 = (processed_Hindi_tokens[i][j])
            max_ratio = 60
            max_ratio_P = 75   #needs to be adjusted
            if (Str1 in EH_dict): # check whether the values exists in english dictionary or not.
                continue
            for l in range(0,len(P_dict)):
                Str2 = P_dict[l][0]
                Ratiostr1 = fuzz.ratio(Str1,Str2)
                if (Ratiostr1 >= max_ratio_P):
                    print(Ratiostr1)
                    max_ratio_P = Ratiostr1
                    flag = 1
                    print(processed_Hindi_tokens[i][j])
                    processed_Hindi_tokens[i][j] = P_dict[l][1]
                    print(f"{flag}-{processed_Hindi_tokens[i][j]}") 
                    break
            for p in EH_dict_F:
                Ratiostr1 = fuzz.ratio(Str1,str(p))
                if(Ratiostr1 >= 98):
                    flag = 1
                    break
            if (flag == 1):
                continue
            else:
                for k in range(0,len(t_dict)):
                    Str2 = t_dict[k][0]
                    Ratiostr1 = fuzz.ratio(Str1,Str2)
                    if (Ratiostr1 > max_ratio):
                        max_ratio = Ratiostr1
                        processed_Hindi_tokens[i][j] = t_dict[k][1]
    print(processed_Hindi_tokens[0])
    print(processed_Hindi_tokens[1])
    print(processed_tokens[12])
    return processed_Hindi_tokens

In [11]:
def translate_hindi_back_to_English(processed_Hindi_tokens, HE_dict, H_dict_F):
    for i in range(0,len(processed_Hindi_tokens)):
        print(i)
        for j in range (0,len(processed_Hindi_tokens[i])):
            Str = processed_Hindi_tokens[i][j]
            if(Str in HE_dict):
                processed_Hindi_tokens[i][j] = HE_dict[Str]
            elif(Str in H_dict_F):
                processed_Hindi_tokens[i][j] = H_dict_F[Str]
    return processed_Hindi_tokens

In [12]:
def load_vectors(fname):
    path = str(fname)
    fin = io.open(path, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [13]:
!pip install fasttext
import fasttext
import fasttext.util
ft = fasttext.load_model('cc.en.300.bin')
ft.get_dimension()
fasttext.util.reduce_model(ft, 200)
ft.get_dimension()

200

In [14]:
processed_tokens, tags = get_processed_tokens_for_english_tweets()
processed_Hindi_tokens = get_processed_hindi_tokens()
t_dict = get_transliteration_Hinglish_Hindi_dict()
P_dict = get_profanity_dict()
HE_dict, H_dict_F, EH_dict, EH_dict_F = translate_hindi_to_english()
processed_Hindi_tokens = get_translated_hindi_english(HE_dict, H_dict_F)
#processed_Hindi_tokens = get_token_translations(processed_tokens, processed_Hindi_tokens, EH_dict, P_dict, t_dict)

11999
11999
First stage par dus jootey khaye Grover  se
['randtv', 'tumhare', 'najayaz', 'baap', 'is', 'area', 'hai', 'ki', 'waha', 'koi', 'nahi', 'has', 'sakta', 'haraami', 'azad', 'mulk', 'hai', 'sab', 'jagah', 'jayenge']
           Hinglish        Hindi
0         hajagiree       हजगिरी
1          chekaanv        चेकॉव
2        spinagaarn   स्पिनगार्न
3             medal         मेडल
4       chetthinaad    चेत्तिनाद
...             ...          ...
14914          roda         रोडा
14915  shymaleshwor  स्यामलेश्वर
14916           bar          वार
14917       leonard    लियोनार्ड
14918      gurudwar   गुरूद्वारा

[14919 rows x 2 columns]
After NP array
[['hajagiree' 'हजगिरी']
 ['chekaanv' 'चेकॉव']
 ['spinagaarn' 'स्पिनगार्न']
 ...
 ['bar' 'वार']
 ['leonard' 'लियोनार्ड']
 ['gurudwar' 'गुरूद्वारा']]
Profanity
       Hinglish         English
0         badir           idiot
1    badirchand           idiot
2       bakland           idiot
3        bhadva            pimp
4     bhootnika  son 

In [ ]:
processed_Hindi_tokens = pd.read_csv("hindi_tokens_translated_to_English_list.csv", sep="\s+")
#processed_Hindi_tokens = translate_hindi_back_to_English(processed_Hindi_tokens, HE_dict, H_dict_F)

In [ ]:
processed_Hindi_tokens

In [40]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

def prepare_sequence_tags(seq, to_ix):
    idxs = to_ix[seq]
    idxs = torch.tensor(idxs, dtype=torch.long)
    idxs = idxs.view(1)
    return idxs

In [16]:
def sentence_to_padded_sentence(sentence,word_to_ix):
    
    # map sentences to vocab
    sentence =  [[word_to_ix[word] for word in sent] for sent in sentence]
    # sentence now looks like:  
    # [[1, 2, 3, 4, 5, 6, 7], [8, 8], [7, 9]]
    sentence_lengths = [len(sent) for sent in sentence]
    pad_token = word_to_ix['<PAD>']
    longest_sent = max(sentence_lengths)
    batch_size = len(sentence)
    padded_sentence = np.ones((batch_size, longest_sent)) * pad_token
    for i, x_len in enumerate(sentence_lengths):
        sequence = sentence[i]
        padded_sentence[i, 0:x_len] = sequence[:x_len]
  
    return padded_sentence

In [59]:
ix_to_word

{}

In [65]:
word_to_ix = {}
ix_to_word = {}
tag_to_ix = {}
ix_to_tag = {}
word_to_ix = {"<PAD>":0}

In [66]:
#training_data = utils.substitute_with_UNK(processed_tokens,1)
training_data = utils.substitute_with_UNK(processed_tokens,word_to_ix)
testing_data = utils.substitute_with_UNK_for_TEST(processed_tokens,word_to_ix)
print(len(training_data))


11999


In [67]:
english_embeddings_fasttext = ft.get_words()

In [68]:
english_embeddings_fasttext

[',',
 'the',
 '.',
 'and',
 'to',
 'of',
 'a',
 '</s>',
 'in',
 'is',
 ':',
 'I',
 'for',
 'that',
 ')',
 '"',
 '(',
 'on',
 'with',
 'it',
 'you',
 'The',
 'was',
 'as',
 'are',
 'at',
 '/',
 '’',
 'be',
 'by',
 "'s",
 'this',
 'have',
 'from',
 'or',
 '!',
 'not',
 'your',
 'an',
 "'",
 'but',
 '?',
 'can',
 '-',
 'will',
 's',
 'my',
 'has',
 'all',
 'we',
 'they',
 'he',
 'his',
 'more',
 'one',
 'about',
 'their',
 "'t",
 'so',
 'which',
 'It',
 'out',
 'up',
 '...',
 'were',
 'had',
 'who',
 'like',
 ';',
 '“',
 'our',
 'would',
 '”',
 'time',
 'been',
 'if',
 'also',
 'just',
 'when',
 'her',
 'This',
 'me',
 'there',
 'do',
 'what',
 'some',
 'other',
 'In',
 'them',
 '–',
 '1',
 'get',
 'new',
 'into',
 '&',
 'We',
 'than',
 'A',
 'no',
 'only',
 'first',
 'any',
 'its',
 'people',
 '2',
 '$',
 'very',
 't',
 'over',
 'she',
 '%',
 'how',
 'make',
 'You',
 'said',
 'He',
 'two',
 'may',
 'know',
 'then',
 'see',
 'after',
 'most',
 'good',
 'years',
 'If',
 'these',
 'now',
 

In [ ]:
english_embeddings_fasttext = [word.lower() for word in english_embeddings_fasttext if not word in stopwords.words('english')]
english_embeddings_fasttext = [word.lower() for word in english_embeddings_fasttext if word.isalpha()]                              

In [69]:
data_vector = dict()
word_vector_list = []

In [ ]:
#For PAD
dummy_list = []
dummy_list = np.zeros(200, dtype = float)
word_vector_list.append(dummy_list)

for i, word in enumerate(english_embeddings_fasttext):
    word_to_ix[word] = len(word_to_ix)
    word_vector_list.append(list(ft.get_word_vector(word)))
for tag in tags:
    if tag not in tag_to_ix:
        tag_to_ix[tag] = len(tag_to_ix)
        ix_to_tag[tag_to_ix[tag]] = tag

word_to_ix["UNK"] = len(word_to_ix)

#For UNK
word_vector_list.append(dummy_list)
word_vector_list = np.asarray(word_vector_list)

In [25]:
# for sent in training_data:
#     for word in sent:
#         if word not in word_to_ix:
#             word_to_ix[word] = len(word_to_ix)
#             ix_to_word[word_to_ix[word]] = word
# for tag in tags:
#     if tag not in tag_to_ix:
#         tag_to_ix[tag] = len(tag_to_ix)
#         ix_to_tag[tag_to_ix[tag]] = tag

# sentence= []
# for sent in training_data:
#      sentence.append(sent[:50])

In [28]:
training_data = utils.substitute_with_UNK_for_TEST(processed_tokens,word_to_ix)
#testing_data = utils.substitute_with_UNK_for_TEST(processed_tokens,word_to_ix)

In [ ]:
test_sentence = []
for sent in testing_data:
     test_sentence.append(sent[:50])

In [30]:
sentence= []
for sent in training_data:
     sentence.append(sent[:50])
padded_sentence = sentence_to_padded_sentence(sentence, word_to_ix)
#test_padded_sentence = sentence_to_padded_sentence(test_sentence, word_to_ix)

In [44]:
class MIMCT(nn.Module):   
    def __init__(self,input_channel,vocab_size,word_to_ix,output_channel,embedding_dim,hidden_dim,kernel_size,feature_linear, word_vector_list):
        super(MIMCT, self).__init__()
        
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.word_embeddings.weight.data.copy_(torch.from_numpy(word_vector_list))
        
        self.CNN_Layers = nn.Sequential( 
            nn.Conv1d(input_channel, output_channel,kernel_size[0], stride=1),
            nn.Conv1d(input_channel, output_channel, kernel_size[1], stride=1),
            nn.Conv1d(input_channel, output_channel, kernel_size[2], stride=1),
            nn.Flatten(),nn.Dropout(p=0.25),
            nn.Linear(feature_linear, 64),
            nn.ReLU(),
            nn.Linear(64, 3),
            nn.Softmax()
            )
        
        
        #create LSTM.
        #self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim,3)
        self.dropout = nn.Dropout(p=0.20)
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=1)
        self.linear = nn.Linear(50+1,3)
    def forward(self,x):
      #  y = self.LSTM_Layers(x)
        print(x)
        embeds = self.word_embeddings(x)
        print(embeds)
        embeds_cnn = embeds.view(1,embeds.size(0),embeds.size(1))
        cnn_output = self.CNN_Layers(embeds_cnn)
        
        lstm_out, _ = self.lstm(embeds.view(len(x), 1, -1))
        lstm_out= self.dropout(lstm_out)
        tag_space = self.hidden2tag(lstm_out.view(len(x), -1))
        
        lstm_output = self.sigmoid(tag_space)
        #concat the outputs the compile layer with categorical cross-entropy the loss function,
        lstm_output = lstm_output.view(lstm_output.size(0),-1)
        cnn_output = cnn_output.view(cnn_output.size(0),-1)
        X = torch.cat((lstm_output,cnn_output))
        X = X.view(1,X.size(0),X.size(1))
        X = self.maxpool(X)
        X = self.linear(X.view(X.size(2), -1))
        X = self.softmax(X)
        print(X)
        return X

In [35]:
batch_size = 1
input_channel = 10 #vocab size
vocab_size = len(word_to_ix) 
embedding_dim = 200 
output_channel = 10
kernel_size = [20,15,10]
Feature_layer1 = embedding_dim - kernel_size[0] + 1
Feature_layer2 = Feature_layer1 - kernel_size[1] + 1
Feature_layer3 = Feature_layer2 - kernel_size[2] + 1
feature_linear = Feature_layer3 * input_channel

In [ ]:
training_data = utils.substitute_with_UNK(processed_tokens,1)
print(len(training_data))

word_to_ix = {}
ix_to_word = {}
tag_to_ix = {}
ix_to_tag = {}
for sent in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            ix_to_word[word_to_ix[word]] = word
for tag in tags:
    if tag not in tag_to_ix:
        tag_to_ix[tag] = len(tag_to_ix)
        ix_to_tag[tag_to_ix[tag]] = tag

In [45]:
batch_size = 1
input_channel = 50 #vocab size
vocab_size = len(word_to_ix) 
embedding_dim = 200 
output_channel = 50
kernel_size = [20,15,10]
Feature_layer1 = embedding_dim - kernel_size[0] + 1
Feature_layer2 = Feature_layer1 - kernel_size[1] + 1
Feature_layer3 = Feature_layer2 - kernel_size[2] + 1
feature_linear = Feature_layer3 * input_channel

#Parameters for LSTM
hidden_dim = 128
dropout = 0.25, 
#recurrent_dropout = 0.3

model = MIMCT(input_channel,vocab_size,word_to_ix,output_channel,embedding_dim,hidden_dim,kernel_size,feature_linear, word_vector_list)
loss_function = nn.CrossEntropyLoss()
#Adam Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)
original_data = training_data
training_data = padded_sentence
# sentence1 = training_data[0]

print(len(tags))

11999


In [37]:
training_data

array([[1.32000e+02, 1.15000e+02, 4.87369e+05, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [7.69000e+02, 3.23400e+03, 3.31050e+04, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [5.83000e+02, 4.39340e+04, 1.34870e+04, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       ...,
       [1.87270e+04, 6.00000e+02, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [8.48000e+02, 1.06280e+04, 1.55940e+04, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [6.39000e+02, 6.80000e+01, 8.76600e+03, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00]])

In [47]:
for epoch in range(5):  # running for 20 epoch
    print(f"Starting epoch {epoch}...")
    #for sentence in training_data:
    for index,sentence in enumerate(training_data):
        print(sentence)
        model.zero_grad()
        targets = tags[index]
        sentence_in=torch.tensor(sentence, dtype=torch.long)
        print(sentence_in)
        targets = prepare_sequence_tags(targets, tag_to_ix)
        print(sentence_in)
        tag_scores = model(sentence_in)
 
        loss = loss_function(tag_scores.cpu(), targets)
        print(loss)
        loss.backward()
        optimizer.step()

Starting epoch 0...
[1.320000e+02 1.150000e+02 4.873690e+05 7.059000e+03 1.313000e+03
 2.000001e+06 3.606000e+04 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00]
tensor([    132,     115,  487369,    7059,    1313, 2000001,   36060,       0,
              0,       0,       0,       0,       0,       0,       0,       0,
              0,       0,       0,       0,       0,       0,       0,       0,
              0,       0,       0,       0,       0,       0,       0,       0,

IndexError: index out of range in self

In [ ]:
processed_Hindi_tokens

import csv

with open("hindi_tokens_translated_to_English_list.csv", "w", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(processed_Hindi_tokens)

In [ ]:
testing_data = test_padded_sentence
with torch.no_grad():
	# this will be the file to write the outputs
    with open("mymodel_output.txt", 'w',encoding='UTF-8') as op:
        for instance in testing_data:
            # Convert the test sentence into a word ID tensor
            test_sentence_in=torch.tensor(instance, dtype=torch.long)

            tag_scores = model(test_sentence_in)

            # Find the tag with the highest probability in each position
            outputs = [int(np.argmax(ts)) for ts in tag_scores.cpu().detach().numpy()]
            # Prepare the output to be written in the same format as the test file (word|tag)
            formatted_output = ix_to_tag[outputs[0]]
            # Write the output
            op.write(formatted_output + '\n')
            
        print(i)
        print(len(test_data))